In [116]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [117]:
# import tensorflow.keras.backend as K
import numpy as np
def rho_risk2(y_true, y_pred, rho):
    #
    # This function calculates the quantile risk between the true and the predicted outputs
    #
    num1 = (y_true-y_pred)*rho*(y_true>=y_pred)
    num2 = (y_pred-y_true)*(1-rho)*(y_true<y_pred)
    den  = np.sum(np.abs(y_true), axis=None)
    
    return 2*(np.sum(num1, axis=None)+np.sum(num2, axis=None))/den

def nrmse(y_true, y_pred):
    #
    # The formula is:
    #           K.sqrt(K.mean(K.square(y_pred - y_true)))    
    #    NRMSE = -----------------------------------------------
    #           K.mean(K.sum(y_true))       
    #
    num = K.sqrt(K.mean(K.square(y_true - y_pred), axis=None))
    den = K.mean(K.abs(y_true), axis=None)
    
    return num / den

def rho_risk(y_true, y_pred, rho):
    #
    # This function calculates the quantile risk between the true and the predicted outputs
    #
    num = -np.sum(2*(y_pred-y_true)*(rho*(y_pred<=y_true)-(1-rho)*(y_pred>y_true)), axis=None)
    den  = np.sum(np.abs(y_true), axis=None)
    
    return num/den

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200 * np.mean(diff)

def ND(y_pred, y_true):
    demoninator = np.sum(np.abs(y_true))
    diff = np.sum(np.abs(y_true - y_pred))
    return 1.0*diff/demoninator

def rmsle(y_pred, y_true) :
    assert len(y_true) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_true))**2))

def NRMSE(y_pred, y_true):
    assert len(y_pred) == len(y_true)
    denominator = np.mean(np.abs(y_true))
    diff = np.sqrt(np.mean(((y_pred-y_true)**2)))
    return diff/denominator

def rho_risk22(y_pred,y_true,rho):
    assert len(y_pred) == len(y_true)
    diff1 = (y_true-y_pred)*rho*(y_true>=y_pred)
    diff2 = (y_pred-y_true)*(1-rho)*(y_true<y_pred)
    denominator = np.sum(y_true)
    return 2*(np.sum(diff1)+np.sum(diff2))/denominator

import scipy.stats as st

def quantize(mean, var, q):
    z_score = st.norm.ppf(q)
    return mean + z_score*var

def coverage(target, quantile_forecast):
    return np.mean((target < quantile_forecast))

def AEC(nominal_coverage, real_coverage):
    return (real_coverage - nominal_coverage)*100

In [118]:
def naive_daily_seasonal(y_true_df, horizon):
    # NOTE: y_true_df with zeros for the horizon
    h = 1
    for i, row in y_true_df.iloc[-horizon:].iterrows():
        if h <=24:
            #print('less than 24')
            y_true_df.loc[i,:] = y_true_df.loc[(i-timedelta(hours=24)),:].values
            #print('24 h back')
        else:
            #print('more than 24')
            y_true_df.loc[i,:] = y_true_df.loc[(i-timedelta(hours=48)),:].values
            #print('48 h back')
        h+=1
        
    return y_true_df.iloc[-horizon:]

In [119]:
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

def naive_weekly_seasonal(y_true_df, horizon):
    # NOTE: y_true_df with zeros for the horizon
    h = 1
    for i, row in y_true_df.iloc[-horizon:].iterrows():
        #print(f'forecasting {i} on {i.dayofweek}')
        if dayOfWeek[i.dayofweek] in ['Monday', 'Saturday', 'Sunday']:
            y_true_df.loc[i,:] = y_true_df.loc[(i-timedelta(hours=168)),:].values
            #print('168 h back')
        elif dayOfWeek[i.dayofweek] in ['Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            if h <=24:
                #print('less than 24')
                y_true_df.loc[i,:] = y_true_df.loc[(i-timedelta(hours=24)),:].values
                #print('24 h back')
            else:
                #print('more than 24')
                if dayOfWeek[i.dayofweek] in ['Wednesday', 'Thursday', 'Friday']:
                    #print('48 h back')
                    y_true_df.loc[i,:] = y_true_df.loc[(i-timedelta(hours=48)),:].values
                elif dayOfWeek[i.dayofweek] in ['Tuesday']:
                    #print('168 h back')
                    y_true_df.loc[i,:] = y_true_df.loc[(i-timedelta(hours=168)),:].values
                else:
                    raise error
        else:
            raise error
            
        h+=1
        
    return y_true_df.iloc[-horizon:]

## Prediction

In [120]:
for dataset in ['electricity', 'europe_power_system']:
    if dataset=='electricity':
        #Electricity 
        csv_path = r'../DeepAR/data/elect/electricity.csv'
        test_start = '2014-05-27 00:00:00'
        test_start_7d_offset = '2014-05-20 00:00:00'
        test_start_14d_offset = '2014-05-13 00:00:00'
        test_end = '2014-12-31 23:00:00'

    elif dataset == 'europe_power_system':
        #Open power system dataset
        csv_path = r'../DeepAR/data/elect/europe_power_system.csv'
        test_start = '2017-06-24 00:00:00'
        test_start_7d_offset = '2017-06-17 00:00:00'
        test_start_14d_offset = '2017-06-10 00:00:00'
        test_end = '2017-11-30 23:00:00'

    Y_true = pd.read_csv(csv_path, sep=",", index_col=0, parse_dates=True, decimal='.').astype(float)
    Y_true.fillna(0, inplace=True)
    Y_true.head()
    
    df_true = Y_true[test_start_14d_offset:].copy()
    
    for naive in ['daily', 'weekly']:
        print(naive)
        for horizon in [24, 36]:
            print(horizon)
            frcst_df = Y_true[test_start_7d_offset:].copy() 
            frcst_df.iloc[:,:] = np.nan 
            for index in pd.date_range(start=(pd.Timestamp(test_start_7d_offset)-timedelta(hours=1)+timedelta(hours=horizon)), 
                                       end=(pd.Timestamp(test_end)+timedelta(hours=(horizon))), freq='{}H'.format(horizon)):
                print(index)
                fake_df = df_true[:index].copy()
                fake_df.iloc[-horizon:,] = 0
                if naive == 'daily':
                    frcst_df.loc[(index-timedelta(hours=(horizon-1))):index,:] = naive_daily_seasonal(fake_df, horizon)
                else:
                    frcst_df.loc[(index-timedelta(hours=(horizon-1))):index,:] = naive_weekly_seasonal(fake_df, horizon)            
            # save mean    
            frcst_df.loc[test_start:test_end,].reset_index().to_csv(f'./save/{naive}_naive_y_hat_mean_{str(horizon)}_{dataset}.csv', index=False)
            
            # calculate std
            frcst_df_std = Y_true[test_start:].copy() 
            frcst_df_std.iloc[:,:] = np.nan            
            df_error = frcst_df - Y_true.loc[test_start_7d_offset:,]
            frcst_df_std.iloc[:,:] = np.array([np.std(df_error.iloc[xi-168:xi,],axis=0) for xi in range(168, df_error.shape[0])])
            frcst_df_std.loc[test_start:test_end,].reset_index().to_csv(f'./save/{naive}_naive_y_hat_std_{str(horizon)}_{dataset}.csv', index=False)

electricity
daily
24
2014-05-20 23:00:00
2014-05-20 00:00:00
2014-05-21 23:00:00
2014-05-21 00:00:00
2014-05-22 23:00:00
2014-05-22 00:00:00
2014-05-23 23:00:00
2014-05-23 00:00:00
2014-05-24 23:00:00
2014-05-24 00:00:00
2014-05-25 23:00:00
2014-05-25 00:00:00
2014-05-26 23:00:00
2014-05-26 00:00:00
2014-05-27 23:00:00
2014-05-27 00:00:00
2014-05-28 23:00:00
2014-05-28 00:00:00
2014-05-29 23:00:00
2014-05-29 00:00:00
2014-05-30 23:00:00
2014-05-30 00:00:00
2014-05-31 23:00:00
2014-05-31 00:00:00
2014-06-01 23:00:00
2014-06-01 00:00:00
2014-06-02 23:00:00
2014-06-02 00:00:00
2014-06-03 23:00:00
2014-06-03 00:00:00
2014-06-04 23:00:00
2014-06-04 00:00:00
2014-06-05 23:00:00
2014-06-05 00:00:00
2014-06-06 23:00:00
2014-06-06 00:00:00
2014-06-07 23:00:00
2014-06-07 00:00:00
2014-06-08 23:00:00
2014-06-08 00:00:00
2014-06-09 23:00:00
2014-06-09 00:00:00
2014-06-10 23:00:00
2014-06-10 00:00:00
2014-06-11 23:00:00
2014-06-11 00:00:00
2014-06-12 23:00:00
2014-06-12 00:00:00
2014-06-13 23:00:00

In [10]:
## Week of the day Naive model
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df = pd.DataFrame(index=Y_true[valid_start:].iloc[7*24:].index, 
                  columns=Y_true[valid_start:].iloc[7*24:].columns)
for i, row in df.iterrows():
    print('old', i)
    if dayOfWeek[i.dayofweek] in ['Monday', 'Saturday', 'Sunday']:
        df.loc[i,:] = Y_true[valid_start:].loc[(i-timedelta(hours=168)),:].values
    elif dayOfWeek[i.dayofweek] in ['Tuesday', 'Wednesday', 'Thursday', 'Friday']:
        df.loc[i,:] = Y_true[valid_start:].loc[(i-timedelta(hours=24)),:].values 
    else:
        raise error
df.head()

if csv_path == r'../DeepAR/data/elect/electricity.csv':
    df_ps = df
    df_error_ps = df.iloc[:,:] - Y_true[valid_start:].iloc[7*24:]
    df_ps.reset_index().to_csv('./save/ps_naive_y_hat.csv', index=False)
    df_error_ps.reset_index().to_csv('./save/ps_naive_y_errors.csv', index=False)
elif csv_path == r'../DeepAR/data/elect/electricity.csv':
    df_el = df
    df_error_el = df_el.iloc[:,:] - Y_true[valid_start:].iloc[7*24:]
    df_el.reset_index().to_csv('./save/el_naive_y_hat.csv', index=False)
    df_error_el.reset_index().to_csv('./save/el_naive_y_errors.csv', index=False)


## Backtesting 

### Week of the day naive model


In [13]:
# Week of the day naive model
df_el = pd.read_csv('./save/el_naive_y_hat.csv', sep=",", index_col=0, parse_dates=True, decimal='.').astype(float)
df_error_el = pd.read_csv('./save/el_naive_y_errors.csv', sep=",", index_col=0, parse_dates=True, decimal='.').astype(float)
df_ps = pd.read_csv('./save/ps_naive_y_hat.csv', sep=",", index_col=0, parse_dates=True, decimal='.').astype(float)
df_error_ps = pd.read_csv('./save/ps_naive_y_errors.csv', sep=",", index_col=0, parse_dates=True, decimal='.').astype(float)

In [20]:
series = {'load':[0,59],
          'price':[59,90],
          'wind':[90, 147],
          'solar':[147, 183]}



datasets = ['electricity', 'europe_power_system']
df_test_metric = pd.DataFrame(columns=['ND', 'NRMSE', 'QL10', 'QL50', 'QL90', 
                                       'C10', 'C50', 'C90', 'AEC10', 'AEC50', 'AEC90'])
horizons = [3, 6, 12, 24, 36]
csv_path = r'../DeepAR/data/elect/'

for dataset in datasets:
    file_path = csv_path + '{}.csv'.format(dataset)
    df = pd.read_csv(file_path, sep=",", index_col=0, parse_dates=True, decimal='.').astype(float)
    df.fillna(0, inplace=True)
    
    if dataset == 'electricity':
        df_error_el = df_error_el.iloc[-(168+5256):,:]
        Y_true = df.iloc[-5256:,:].values
        Y_hat_mean = df_el.iloc[-5256:,:]
        Y_hat_std = Y_hat_mean.copy()
        Y_hat_std.iloc[:,:] = np.array([np.std(df_error_el.iloc[xi-168:xi],axis=0) for xi in range(168, df_error_el.shape[0])])
        Y_hat_std = Y_hat_std.iloc[-5256:,:]
        
        #Y_hat_std.reset_index().to_csv(f'./save/el_naive2_y_hat_std.csv', index=False)
        #Y_hat_mean.reset_index().to_csv(f'./save/el_naive2_y_hat.csv', index=False)
        
    else:
        df_error_ps = df_error_ps.iloc[-(168+3816):,:]
        Y_true = df.iloc[-3816:,:].values
        Y_hat_mean = df_ps.iloc[-3816:,:]
        Y_hat_std = Y_hat_mean.copy()
        Y_hat_std.iloc[:,:] = np.array([np.std(df_error_ps.iloc[xi-168:xi],axis=0) for xi in range(168, df_error_ps.shape[0])])
        Y_hat_std = Y_hat_std.iloc[-3816:,:]
        
        #Y_hat_std.reset_index().to_csv(f'./save/ps_naive2_y_hat_std.csv', index=False)
        #Y_hat_mean.reset_index().to_csv(f'./save/ps_naive2_y_hat.csv', index=False)
    
    Y_hat_std = Y_hat_std.astype(float).values
    Y_hat_mean = Y_hat_mean.astype(float).values
    
    for horizon in horizons:

        folder_path = dataset[-12:] + '_%d' % horizon
        
        #Y_true = df
        #Y_hat_mean = df.shift(24)#.dropna(axis=0)
        #df_error = (df - Y_hat_mean).iloc[-5500:,:]
        #Y_hat_std = np.array([np.std(df_error.iloc[xi-168:xi],0) for xi in range(168, df_error.shape[0])])
        
        _nd = ND(Y_hat_mean, Y_true)
        _nrmse = NRMSE(Y_hat_mean, Y_true)
        _pho_risk1 = rho_risk2(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.1), 0.1)
        _pho_risk5 = rho_risk2(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.5), 0.5)
        _pho_risk9 = rho_risk2(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.9), 0.9)
        _cov1 = coverage(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.1))
        _cov5 = coverage(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.5))
        _cov9 = coverage(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.9))
        _aec1 = AEC(0.1, _cov1)
        _aec5 = AEC(0.5, _cov5)
        _aec9 = AEC(0.9, _cov9)
        print('Loss metric %s %d ND: %f, NRMSE: %f, Ql10: %f, Ql50: %f, Ql90: %f, C10: %f, C50: %f, C90: %f' % (dataset,
                                                                                     horizon,
                                                                                     _nd,
                                                                                     _nrmse,
                                                                                     _pho_risk1,
                                                                                     _pho_risk5,
                                                                                     _pho_risk9,
                                                                                    _cov1,
                                                                                    _cov5,
                                                                                    _cov9))
        df_test_metric.loc[folder_path] = [_nd, 
                                           _nrmse,
                                           _pho_risk1,
                                           _pho_risk5,
                                           _pho_risk9,
                                           _cov1,
                                           _cov5,
                                           _cov9,
                                           _aec1,
                                           _aec5,
                                           _aec9]
        if dataset != 'electricity':
            for key in series:
                y_true = Y_true[:,series[key][0]:series[key][1]]
                y_pred = Y_hat_mean[:,series[key][0]:series[key][1]]
                y_std = Y_hat_std[:,series[key][0]:series[key][1]]
                
                _nd = ND(y_pred, y_true)
                _nrmse = NRMSE(y_pred, y_true)
                _pho_risk1 = rho_risk2(y_true, quantize(y_pred, y_std, 0.1), 0.1)
                _pho_risk5 = rho_risk2(y_true, quantize(y_pred, y_std, 0.5), 0.5)
                _pho_risk9 = rho_risk2(y_true, quantize(y_pred, y_std, 0.9), 0.9)
                _cov1 = coverage(y_true, quantize(y_pred, y_std, 0.1))
                _cov5 = coverage(y_true, quantize(y_pred, y_std, 0.5))
                _cov9 = coverage(y_true, quantize(y_pred, y_std, 0.9))
                _aec1 = AEC(0.1, _cov1)
                _aec5 = AEC(0.5, _cov5)
                _aec9 = AEC(0.9, _cov9)
                print('Loss metric %s %d ND: %f, NRMSE: %f, Ql10: %f, Ql50: %f, Ql90: %f, C10: %f, C50: %f, C90: %f' % (dataset,
                                                                                             horizon,
                                                                                             _nd,
                                                                                             _nrmse,
                                                                                             _pho_risk1,
                                                                                             _pho_risk5,
                                                                                             _pho_risk9,
                                                                                            _cov1,
                                                                                            _cov5,
                                                                                            _cov9))
                df_test_metric.loc[folder_path + '_' + key] = [_nd, 
                                                             _nrmse,
                                                             _pho_risk1,
                                                             _pho_risk5,
                                                             _pho_risk9,
                                                             _cov1,
                                                             _cov5,
                                                             _cov9,
                                                             _aec1,
                                                             _aec5,
                                                             _aec9]

# df_test_metric.reset_index().rename(columns={'index':'Scenario'}).to_csv('./save/test_metrics_naive_weekoftheday.csv', index=False)
df_test_metric.head()

Loss metric electricity 3 ND: 0.082980, NRMSE: 0.947762, Ql10: 0.048925, Ql50: 0.082980, Ql90: 0.051533, C10: 0.084576, C50: 0.460856, C90: 0.896302
Loss metric electricity 6 ND: 0.082980, NRMSE: 0.947762, Ql10: 0.048925, Ql50: 0.082980, Ql90: 0.051533, C10: 0.084576, C50: 0.460856, C90: 0.896302
Loss metric electricity 12 ND: 0.082980, NRMSE: 0.947762, Ql10: 0.048925, Ql50: 0.082980, Ql90: 0.051533, C10: 0.084576, C50: 0.460856, C90: 0.896302
Loss metric electricity 24 ND: 0.082980, NRMSE: 0.947762, Ql10: 0.048925, Ql50: 0.082980, Ql90: 0.051533, C10: 0.084576, C50: 0.460856, C90: 0.896302
Loss metric electricity 36 ND: 0.082980, NRMSE: 0.947762, Ql10: 0.048925, Ql50: 0.082980, Ql90: 0.051533, C10: 0.084576, C50: 0.460856, C90: 0.896302
Loss metric europe_power_system 3 ND: 0.122293, NRMSE: 0.380619, Ql10: 0.058825, Ql50: 0.122293, Ql90: 0.062275, C10: 0.095802, C50: 0.440040, C90: 0.885813
Loss metric europe_power_system 3 ND: 0.036504, NRMSE: 0.090875, Ql10: 0.018758, Ql50: 0.036504

,ND,NRMSE,QL10,QL50,QL90,C10,C50,C90,AEC10,AEC50,AEC90
electricity_3,0.08298,0.947762,0.048925,0.08298,0.051533,0.084576,0.460856,0.896302,-1.542376,-3.914411,-0.369813
electricity_6,0.08298,0.947762,0.048925,0.08298,0.051533,0.084576,0.460856,0.896302,-1.542376,-3.914411,-0.369813
electricity_12,0.08298,0.947762,0.048925,0.08298,0.051533,0.084576,0.460856,0.896302,-1.542376,-3.914411,-0.369813
electricity_24,0.08298,0.947762,0.048925,0.08298,0.051533,0.084576,0.460856,0.896302,-1.542376,-3.914411,-0.369813
electricity_36,0.08298,0.947762,0.048925,0.08298,0.051533,0.084576,0.460856,0.896302,-1.542376,-3.914411,-0.369813


### Naive model 1 

In [3]:
series = {'load':[0,59],
          'price':[59,90],
          'wind':[90, 147],
          'solar':[147, 183]}

datasets = ['electricity', 'europe_power_system']
df_test_metric = pd.DataFrame(columns=['ND', 'NRMSE', 'QL10', 'QL50', 'QL90', 
                                       'C10', 'C50', 'C90', 'AEC10', 'AEC50', 'AEC90'])
horizons = [3, 6, 12, 24, 36]
csv_path = r'../DeepAR/data/elect/'

for dataset in datasets:
    file_path = csv_path + '{}.csv'.format(dataset)
    df = pd.read_csv(file_path, sep=",", index_col=0, parse_dates=True, decimal='.').astype(float)
    df.fillna(0, inplace=True)
    Y_pred = df.shift(24).dropna()
    
    if dataset == 'electricity':
        Y_pred.iloc[-(5256):,:].reset_index().to_csv(f'./save/el_naive1_y_hat.csv', index=False)
        Y_true = df.iloc[-(5256+168):,:]
        Y_hat_mean = Y_pred.iloc[-(5256+168):,:]
        Y_error = Y_true.iloc[:,:] - Y_hat_mean.iloc[:,:]
        Y_hat_std = Y_hat_mean.iloc[-(5256):,:].copy()
        Y_hat_std.iloc[:,:] = np.array([np.std(Y_error.iloc[xi-168:xi,:],axis=0) for xi in range(168, Y_error.shape[0])])
        Y_hat_std.reset_index().to_csv(f'./save/el_naive1_y_hat_std.csv', index=False)
        Y_hat_std = Y_hat_std.values[-5256:,:]
        Y_true = Y_true.values[-5256:,:]
        Y_hat_mean = Y_hat_mean.values[-5256:,:]
        
    else:
        Y_pred.iloc[-(3816):,:].reset_index().to_csv(f'./save/ps_naive1_y_hat.csv', index=False)
        Y_true = df.iloc[-(3816+168):,:]
        Y_hat_mean = Y_pred.iloc[-(3816+168):,:]
        Y_error = Y_true.iloc[:,:] - Y_hat_mean.iloc[:,:]
        Y_hat_std = Y_hat_mean.iloc[-(3816):,:].copy()
        print(np.array([np.std(Y_error.iloc[xi-168:xi,:],axis=0) for xi in range(168, Y_error.shape[0])]).shape)
        Y_hat_std.iloc[:,:] = np.array([np.std(Y_error.iloc[xi-168:xi,:],axis=0) for xi in range(168, Y_error.shape[0])])
        Y_hat_std.reset_index().to_csv(f'./save/ps_naive1_y_hat_std.csv', index=False)
        Y_hat_mean = Y_hat_mean.values[-3816:,:]
        Y_true = Y_true.values[-3816:,:]
        Y_hat_std = Y_hat_std.values[-3816:,:]

    for horizon in horizons:

        folder_path = dataset[-12:] + '_%d' % horizon
        
        #Y_true = df
        #Y_hat_mean = df.shift(24)#.dropna(axis=0)
        #df_error = (df - Y_hat_mean).iloc[-5500:,:]
        #Y_hat_std = np.array([np.std(df_error.iloc[xi-168:xi],0) for xi in range(168, df_error.shape[0])])
        
        _nd = ND(Y_hat_mean, Y_true)
        _nrmse = NRMSE(Y_hat_mean, Y_true)
        _pho_risk1 = rho_risk2(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.1), 0.1)
        _pho_risk5 = rho_risk2(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.5), 0.5)
        _pho_risk9 = rho_risk2(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.9), 0.9)
        _cov1 = coverage(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.1))
        _cov5 = coverage(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.5))
        _cov9 = coverage(Y_true, quantize(Y_hat_mean, Y_hat_std, 0.9))
        _aec1 = AEC(0.1, _cov1)
        _aec5 = AEC(0.5, _cov5)
        _aec9 = AEC(0.9, _cov9)
        print('Loss metric %s %d ND: %f, NRMSE: %f, Ql10: %f, Ql50: %f, Ql90: %f, C10: %f, C50: %f, C90: %f' % (dataset,
                                                                                     horizon,
                                                                                     _nd,
                                                                                     _nrmse,
                                                                                     _pho_risk1,
                                                                                     _pho_risk5,
                                                                                     _pho_risk9,
                                                                                    _cov1,
                                                                                    _cov5,
                                                                                    _cov9))
        df_test_metric.loc[folder_path] = [_nd, 
                                           _nrmse,
                                           _pho_risk1,
                                           _pho_risk5,
                                           _pho_risk9,
                                           _cov1,
                                           _cov5,
                                           _cov9,
                                           _aec1,
                                           _aec5,
                                           _aec9]
        if dataset != 'electricity':
            for key in series:
                y_true = Y_true[:,series[key][0]:series[key][1]]
                y_pred = Y_hat_mean[:,series[key][0]:series[key][1]]
                y_std = Y_hat_std[:,series[key][0]:series[key][1]]
                
                _nd = ND(y_pred, y_true)
                _nrmse = NRMSE(y_pred, y_true)
                _pho_risk1 = rho_risk2(y_true, quantize(y_pred, y_std, 0.1), 0.1)
                _pho_risk5 = rho_risk2(y_true, quantize(y_pred, y_std, 0.5), 0.5)
                _pho_risk9 = rho_risk2(y_true, quantize(y_pred, y_std, 0.9), 0.9)
                _cov1 = coverage(y_true, quantize(y_pred, y_std, 0.1))
                _cov5 = coverage(y_true, quantize(y_pred, y_std, 0.5))
                _cov9 = coverage(y_true, quantize(y_pred, y_std, 0.9))
                _aec1 = AEC(0.1, _cov1)
                _aec5 = AEC(0.5, _cov5)
                _aec9 = AEC(0.9, _cov9)
                print('Loss metric %s %d ND: %f, NRMSE: %f, Ql10: %f, Ql50: %f, Ql90: %f, C10: %f, C50: %f, C90: %f' % (dataset,
                                                                                             horizon,
                                                                                             _nd,
                                                                                             _nrmse,
                                                                                             _pho_risk1,
                                                                                             _pho_risk5,
                                                                                             _pho_risk9,
                                                                                            _cov1,
                                                                                            _cov5,
                                                                                            _cov9))
                df_test_metric.loc[folder_path + '_' + key] = [_nd, 
                                                             _nrmse,
                                                             _pho_risk1,
                                                             _pho_risk5,
                                                             _pho_risk9,
                                                             _cov1,
                                                             _cov5,
                                                             _cov9,
                                                             _aec1,
                                                             _aec5,
                                                             _aec9]

#
# df_test_metric.reset_index().rename(columns={'index':'Scenario'}).to_csv('./save/test_metrics_naive.csv', index=False)
df_test_metric.head()

Loss metric electricity 3 ND: 0.076277, NRMSE: 0.657230, Ql10: 0.045110, Ql50: 0.076277, Ql90: 0.045269, C10: 0.085670, C50: 0.480911, C90: 0.908378
Loss metric electricity 6 ND: 0.076277, NRMSE: 0.657230, Ql10: 0.045110, Ql50: 0.076277, Ql90: 0.045269, C10: 0.085670, C50: 0.480911, C90: 0.908378
Loss metric electricity 12 ND: 0.076277, NRMSE: 0.657230, Ql10: 0.045110, Ql50: 0.076277, Ql90: 0.045269, C10: 0.085670, C50: 0.480911, C90: 0.908378
Loss metric electricity 24 ND: 0.076277, NRMSE: 0.657230, Ql10: 0.045110, Ql50: 0.076277, Ql90: 0.045269, C10: 0.085670, C50: 0.480911, C90: 0.908378
Loss metric electricity 36 ND: 0.076277, NRMSE: 0.657230, Ql10: 0.045110, Ql50: 0.076277, Ql90: 0.045269, C10: 0.085670, C50: 0.480911, C90: 0.908378
(3816, 183)
Loss metric europe_power_system 3 ND: 0.139750, NRMSE: 0.421853, Ql10: 0.067498, Ql50: 0.139750, Ql90: 0.081342, C10: 0.088646, C50: 0.462929, C90: 0.901957
Loss metric europe_power_system 3 ND: 0.071248, NRMSE: 0.195388, Ql10: 0.035638, Ql

,ND,NRMSE,QL10,QL50,QL90,C10,C50,C90,AEC10,AEC50,AEC90
electricity_3,0.076277,0.65723,0.04511,0.076277,0.045269,0.08567,0.480911,0.908378,-1.432963,-1.908929,0.837826
electricity_6,0.076277,0.65723,0.04511,0.076277,0.045269,0.08567,0.480911,0.908378,-1.432963,-1.908929,0.837826
electricity_12,0.076277,0.65723,0.04511,0.076277,0.045269,0.08567,0.480911,0.908378,-1.432963,-1.908929,0.837826
electricity_24,0.076277,0.65723,0.04511,0.076277,0.045269,0.08567,0.480911,0.908378,-1.432963,-1.908929,0.837826
electricity_36,0.076277,0.65723,0.04511,0.076277,0.045269,0.08567,0.480911,0.908378,-1.432963,-1.908929,0.837826
